In [12]:
import pandas as pd
import numpy as np
gdp_data=pd.read_excel("C:\\Users\\91978\\Desktop\\inflation_prediction_project\\excel_files\\gdp_changeqtr.xlsx")

gdp_data.head()

,Quarter,GDP % Change
0,1996 Q3,0.5
1,1996 Q4,1.6
2,1997 Q1,0.8
3,1997 Q2,0.4
4,1997 Q3,1.2


In [14]:
# Function to impute quarterly data to monthly
def impute_quarterly_to_monthly(quarter_data):
    # Convert 'Quarter' column to string type
    quarter_data['Quarter'] = quarter_data['Quarter'].astype(str)
    
    # Extract year and quarter from the 'Quarter' column
    quarter_data[['Year', 'Quarter']] = quarter_data['Quarter'].str.split(' Q', expand=True)
    
    # Convert Quarter to numeric
    quarter_data['Quarter'] = quarter_data['Quarter'].astype(int)

    # Create a new DataFrame to store monthly data
    monthly_data = pd.DataFrame(columns=['Year', 'Month', 'GDP Change (%)'])

    # Iterate through each quarter
    for index, row in quarter_data.iterrows():
        # Calculate monthly growth rate
        monthly_growth_rate = row['GDP Change (%)'] / 3
        
        # Distribute the quarterly GDP change across the months
        for month in range(1, 4):
            month_data = {
                'Year': row['Year'],
                'Month': (row['Quarter'] - 1) * 3 + month,
                'GDP Change (%)': monthly_growth_rate
            }
            monthly_data = monthly_data.append(month_data, ignore_index=True)

    return monthly_data

# Impute quarterly data to monthly
monthly_data = impute_quarterly_to_monthly(gdp_data)

# Convert Month and Year to mm/yyyy format
monthly_data['Date'] = pd.to_datetime(monthly_data[['Year', 'Month']].astype(str).agg('-'.join, axis=1).add('-01'))

# Drop Year and Month columns
monthly_data = monthly_data.drop(['Year', 'Month'], axis=1)

# Reorder columns
monthly_data = monthly_data[['Date', 'GDP Change (%)']]

# Print the resulting DataFrame
print(monthly_data)

AttributeError: Can only use .str accessor with string values!